## 1. Perceptron

<img src="./img/perceptron-6168423.jpg" alt="drawing" width="650"/>

Empezamos cargando librerias

In [1]:
# =============================================================================
# IMPORTACIÓN DE LIBRERÍAS
# =============================================================================
# NumPy: Librería fundamental para cálculos numéricos y operaciones con arrays
import numpy as np

# Pandas: Herramienta para manipulación y análisis de datos estructurados
import pandas as pd

# Seaborn: Librería de visualización de datos basada en matplotlib
# También incluye datasets de ejemplo como el de pingüinos que usaremos
import seaborn as sns

Cargamos datos. Utilizaremos el dataset de pinguinos de seaborn

In [2]:
# =============================================================================
# CARGA DEL DATASET DE PINGÜINOS
# =============================================================================
# Cargamos el dataset de pingüinos de Palmer, que incluye mediciones
# de 3 especies diferentes de pingüinos (Adelie, Chinstrap, Gentoo)
df = sns.load_dataset("penguins")

# Visualizamos las primeras 5 filas para entender la estructura de los datos
df.head()

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,Male
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,Female
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,Female
3,Adelie,Torgersen,NaN,NaN,NaN,NaN,NaN
4,Adelie,Torgersen,36.7,19.3,193.0,3450.0,Female


In [3]:
# =============================================================================
# INFORMACIÓN DEL DATASET
# =============================================================================
# Usamos .info() para ver:
# - El número total de registros (344 pingüinos)
# - Los tipos de datos de cada columna
# - Valores nulos (importante: hay algunos NaN en las columnas numéricas y en 'sex')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 344 entries, 0 to 343
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   species            344 non-null    object 
 1   island             344 non-null    object 
 2   bill_length_mm     342 non-null    float64
 3   bill_depth_mm      342 non-null    float64
 4   flipper_length_mm  342 non-null    float64
 5   body_mass_g        342 non-null    float64
 6   sex                333 non-null    object 
dtypes: float64(4), object(3)
memory usage: 18.9+ KB


In [4]:
# =============================================================================
# LIMPIEZA Y PREPROCESAMIENTO DE DATOS
# =============================================================================
# Volvemos a cargar el dataset
df = sns.load_dataset("penguins")

# PASO 1: Eliminar filas con valores nulos
# inplace=True modifica el DataFrame directamente sin crear una copia
df.dropna(inplace=True)

# PASO 2: Codificación de variables categóricas a numéricas
# Las redes neuronales necesitan datos numéricos, no texto
cleanup_nums = {"species": {"Adelie": 0,      # Especie Adelie -> 0
                            "Chinstrap": 1,   # Especie Chinstrap -> 1
                            "Gentoo": 2},     # Especie Gentoo -> 2
               "sex": {"Male": 0,             # Macho -> 0
                       "Female": 1}}          # Hembra -> 1

# Aplicamos el mapeo de valores categóricos a numéricos
df.replace(cleanup_nums, inplace=True)

# PASO 3: One-Hot Encoding para la variable 'island'
# get_dummies() convierte variables categóricas en columnas binarias (0 o 1)
# Por ejemplo: 'island' se convierte en 'island_Biscoe', 'island_Dream', 'island_Torgersen'
df = pd.get_dummies(df)

# Mostramos el resultado del preprocesamiento
df.head()

C:\Users\borja\AppData\Local\Temp\ipykernel_10800\2354517574.py:20: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.replace(cleanup_nums, inplace=True)


,species,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex,island_Biscoe,island_Dream,island_Torgersen
0,0,39.1,18.7,181.0,3750.0,0,False,False,True
1,0,39.5,17.4,186.0,3800.0,1,False,False,True
2,0,40.3,18.0,195.0,3250.0,1,False,False,True
4,0,36.7,19.3,193.0,3450.0,1,False,False,True
5,0,39.3,20.6,190.0,3650.0,0,False,False,True


In [5]:
# =============================================================================
# VERIFICACIÓN DESPUÉS DEL PREPROCESAMIENTO
# =============================================================================
# Comprobamos que:
# - Ahora tenemos 333 registros (eliminamos los que tenían NaN)
# - Todas las columnas son numéricas
# - Las islas se han convertido en 3 columnas binarias (one-hot encoding)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 333 entries, 0 to 343
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   species            333 non-null    int64  
 1   bill_length_mm     333 non-null    float64
 2   bill_depth_mm      333 non-null    float64
 3   flipper_length_mm  333 non-null    float64
 4   body_mass_g        333 non-null    float64
 5   sex                333 non-null    int64  
 6   island_Biscoe      333 non-null    bool   
 7   island_Dream       333 non-null    bool   
 8   island_Torgersen   333 non-null    bool   
dtypes: bool(3), float64(4), int64(2)
memory usage: 19.2 KB


In [6]:
# =============================================================================
# ESTADÍSTICAS DESCRIPTIVAS
# =============================================================================
# describe() nos muestra estadísticas como media, desviación estándar, 
# mínimo, máximo y cuartiles de cada variable numérica
# Esto es importante para entender la escala de nuestras variables
df.describe()

,species,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex
count,333.000000,333.000000,333.000000,333.000000,333.000000,333.000000
mean,0.918919,43.992793,17.164865,200.966967,4207.057057,0.495495
std,0.889718,5.468668,1.969235,14.015765,805.215802,0.500732
min,0.000000,32.100000,13.100000,172.000000,2700.000000,0.000000
25%,0.000000,39.500000,15.600000,190.000000,3550.000000,0.000000
50%,1.000000,44.500000,17.300000,197.000000,4050.000000,0.000000
75%,2.000000,48.600000,18.700000,213.000000,4775.000000,1.000000
max,2.000000,59.600000,21.500000,231.000000,6300.000000,1.000000


Dividimos en train test

In [7]:
# =============================================================================
# DIVISIÓN EN TRAIN Y TEST
# =============================================================================
from sklearn.model_selection import train_test_split

# Separamos las características (X) de la variable objetivo (y)
# iloc[:, 1:] selecciona todas las filas y desde la columna 1 en adelante (features)
X = df.iloc[:, 1:]  # Características: todas las columnas excepto 'species'

# iloc[:, 0] selecciona la primera columna (species), nuestra variable objetivo
y = df.iloc[:, 0]   # Variable objetivo: especie del pingüino (0, 1 o 2)

# Dividimos los datos en conjuntos de entrenamiento (80%) y prueba (20%)
# test_size=0.2 significa que el 20% de los datos se usarán para test
# random_state=42 asegura que la división sea reproducible
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=0.2,
                                                    random_state=42)

In [8]:
# =============================================================================
# VERIFICACIÓN DE LAS DIMENSIONES
# =============================================================================
# Imprimimos las dimensiones de cada conjunto para confirmar la división
print(X_train.shape)  # (266, 8) -> 266 muestras de entrenamiento, 8 características
print(X_test.shape)   # (67, 8) -> 67 muestras de test, 8 características
print(y_train.shape)  # (266,) -> 266 etiquetas de entrenamiento
print(y_test.shape)   # (67,) -> 67 etiquetas de test

(266, 8)
(67, 8)
(266,)
(67,)


Vamos a probar un Perceptrón

In [9]:
# =============================================================================
# PRIMER MODELO: PERCEPTRÓN SIMPLE SIN ESTANDARIZAR
# =============================================================================
from sklearn.linear_model import Perceptron

# Creamos un Perceptrón (red neuronal de una sola capa)
# El perceptrón es el modelo más simple de red neuronal
per_clf = Perceptron(random_state=1)

# Entrenamos el modelo con los datos de entrenamiento
per_clf.fit(X_train, y_train)

# Evaluamos el modelo en el conjunto de test
# score() devuelve la precisión (accuracy): % de predicciones correctas
# RESULTADO: ~19% de precisión - ¡MUY MALO! Peor que adivinar al azar
per_clf.score(X_test, y_test)

0.19402985074626866

In [10]:
# =============================================================================
# COMPARACIÓN: REGRESIÓN LOGÍSTICA SIN ESTANDARIZAR
# =============================================================================
from sklearn.linear_model import LogisticRegression

# Probamos con Regresión Logística para comparar
# max_iter=10000 establece el número máximo de iteraciones para convergencia
log_reg = LogisticRegression(max_iter=10000)

# Entrenamos el modelo
log_reg.fit(X_train, y_train)

# Evaluamos en el conjunto de test
# RESULTADO: ~98.5% de precisión - ¡EXCELENTE!
# Esto demuestra que el Perceptrón simple tiene problemas con estos datos
log_reg.score(X_test, y_test)

1.0

Probemos a estandarizar

Parece que el perceptrón por si solo es bastante inútil, habrá que probar configuraciones más complejas.

## 2. Multi Layer Perceptron

In [11]:
# =============================================================================
# MULTI LAYER PERCEPTRON (MLP) - CONFIGURACIÓN POR DEFECTO
# =============================================================================
from sklearn.neural_network import MLPClassifier
# También existe MLPRegressor para problemas de regresión

# Creamos un MLP con configuración por defecto
# Por defecto tiene una capa oculta con 100 neuronas
mlp = MLPClassifier(random_state=42)

# Entrenamos el modelo
mlp.fit(X_train, y_train)

# Evaluamos en el conjunto de test
# RESULTADO: ~43% de precisión - Mejor que el Perceptrón simple, pero aún malo
# El MLP no está funcionando bien sin estandarización
mlp.score(X_test, y_test)

0.43283582089552236

Probemos otra configuración. Es posible crear una red neuronal desde la propia función de MLPClassifier()

In [15]:
# =============================================================================
# MLP CON CONFIGURACIÓN PERSONALIZADA (SIN ESTANDARIZAR)
# =============================================================================
# Intentamos mejorar el MLP con una arquitectura más compleja
mlp = MLPClassifier(max_iter=500,                      # Aumentamos iteraciones
                   activation='relu',                  # Función de activación ReLU
                   hidden_layer_sizes = (150, 150, 150),  # 3 capas ocultas con 150 neuronas cada una
                   random_state=42)

# Entrenamos el modelo con la nueva arquitectura
mlp.fit(X_train, y_train)

# Evaluamos en el conjunto de test
# RESULTADO: ~46% de precisión - Apenas mejoró
# CONCLUSIÓN: El problema NO es la arquitectura, es la FALTA DE ESTANDARIZACIÓN
mlp.score(X_test, y_test)

0.4626865671641791

Utilizan descenso del gradiente, y por tanto son muy sensibles al escalado. Estandarizamos para el siguiente ejemplo

In [16]:
# =============================================================================
# ESTANDARIZACIÓN DE DATOS + PERCEPTRÓN
# =============================================================================
# Las redes neuronales utilizan descenso del gradiente para optimización
# Este algoritmo es MUY SENSIBLE a la escala de las características
# Por ejemplo: body_mass_g está en miles, pero sex está entre 0 y 1

from sklearn.preprocessing import StandardScaler

# PASO 1: Crear el escalador
# StandardScaler transforma los datos para que tengan media=0 y desviación estándar=1
sc = StandardScaler()

# PASO 2: Ajustar el escalador SOLO con los datos de entrenamiento
# fit() calcula la media y desviación estándar de X_train
sc.fit(X_train)

# PASO 3: Transformar tanto train como test usando las estadísticas de train
# ¡IMPORTANTE! Usamos las mismas estadísticas para evitar data leakage
X_train_s = sc.transform(X_train)  # Estandarizamos el conjunto de entrenamiento
X_test_s = sc.transform(X_test)    # Estandarizamos el conjunto de test

# PASO 4: Entrenar Perceptrón con datos estandarizados
per_clf = Perceptron()
per_clf.fit(X_train_s, y_train)

# PASO 5: Evaluar el modelo
print(per_clf.score(X_train_s, y_train))  # RESULTADO: 100% en train
print(per_clf.score(X_test_s, y_test))    # RESULTADO: 100% en test
# ¡INCREÍBLE MEJORA! De 19% a 100% solo con estandarización

1.0
1.0


In [17]:
# =============================================================================
# REGRESIÓN LOGÍSTICA CON DATOS ESTANDARIZADOS
# =============================================================================
# Probamos también la Regresión Logística con datos estandarizados
log_reg = LogisticRegression(max_iter=500)

# Entrenamos con los datos estandarizados
log_reg.fit(X_train_s, y_train)

# Evaluamos en ambos conjuntos
print(log_reg.score(X_train_s, y_train))  # 100% en train
print(log_reg.score(X_test_s, y_test))    # 100% en test
# La Regresión Logística ya funcionaba bien, pero la estandarización la hace perfecta

1.0
1.0


In [18]:
# =============================================================================
# MLP CON ESTANDARIZACIÓN - SOLUCIÓN DEFINITIVA
# =============================================================================
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler

# PASO 1: Crear y ajustar el escalador
scaler = StandardScaler()
scaler.fit(X_train)

# PASO 2: Transformar los datos
X_train_scal = scaler.transform(X_train)
X_test_scal = scaler.transform(X_test)

# PASO 3: Crear y entrenar el MLP (con configuración por defecto)
mlp = MLPClassifier(max_iter=500)
mlp.fit(X_train_scal, y_train)

# PASO 4: Evaluar el modelo
print(mlp.score(X_train_scal, y_train))  # RESULTADO: 100% en train
print(mlp.score(X_test_scal, y_test))    # RESULTADO: 100% en test

# LECCIÓN CLAVE: La estandarización es CRÍTICA para redes neuronales
# Sin ella: 43% de precisión
# Con ella: 100% de precisión

1.0
1.0


In [19]:
# =============================================================================
# MATRIZ DE CONFUSIÓN - ANÁLISIS DETALLADO DE RESULTADOS
# =============================================================================
from sklearn.metrics import confusion_matrix

# La matriz de confusión muestra cómo se distribuyen las predicciones
# Filas: clases reales | Columnas: clases predichas
# Diagonal principal: predicciones correctas
confusion_matrix(y_test, mlp.predict(X_test_scal))

# INTERPRETACIÓN DEL RESULTADO:
# [[31  0  0]   <- 31 Adelie correctamente clasificados, 0 errores
#  [ 0 13  0]   <- 13 Chinstrap correctamente clasificados, 0 errores
#  [ 0  0 23]]  <- 23 Gentoo correctamente clasificados, 0 errores
# ¡PERFECTO! No hay errores de clasificación

array([[31,  0,  0],
       [ 0, 13,  0],
       [ 0,  0, 23]])